In [122]:
import cv2
import numpy as np

# Detection Methods

In [123]:
def drawCenter(cx , cy , frame):
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
    
def getMaxContourCenter(processedImage , frame):
    contours, _  = cv2.findContours(processedImage, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)==0):
        return None
    max_contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(max_contour)
    try:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        drawCenter(cx , cy , frame)
        for contour in contours:
            c = contour
            if(c is max_contour):
                cv2.drawContours(frame, c, -1, (0, 255 , 0 ), 3)
            else: 
                cv2.drawContours(frame, c, -1, (0, 0, 255), 3)
        return cx , cy
    except ZeroDivisionError:
        print("Zero Division Error")
        return None

# Global Config

In [124]:
kernel_size = 5
low_threshold = 10
high_threshold = 50

### Method of Canny

In [125]:
def canny(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    
    # Apply Canny edge detection
    # edges = cv2.Canny(blur, low_threshold, high_threshold)

    edges = cv2.Laplacian(blur, -1, ksize=5)

    edges = cv2.medianBlur(edges,9)

    # edges = cv2.GaussianBlur(edges, (kernel_size, kernel_size), 0)


    kernal = np.ones((3, 3), np.uint8)
    # edges = cv2.dilate(edges, kernal, iterations=10)
    edges = cv2.erode(edges, kernal, iterations=1)
    # edges = cv2.dilate(edges, kernal, iterations=10)

    cv2.imshow('canny', edges)

    return getMaxContourCenter(edges , frame)

### Method of houghLines

In [126]:
rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 15  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 100  # minimum number of pixels making up a line
max_line_gap = 50  # maximum gap in pixels between connectable line segments

In [127]:
def houghLines(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)


    line_image = np.copy(frame) * 0  # creating a blank to draw lines on
    if lines is None:
        return None
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
    
    cv2.imshow('hough', line_image)

### Using color Detection

In [128]:
def detect_black(frame):

    black_line = cv2.inRange(frame, (0, 0, 0), (40, 40, 40))

    kernel = np.ones((3, 3), np.uint8)
    black_line = cv2.dilate(black_line, kernel, iterations=2)

    cv2.imshow("black_line", black_line)
    return getMaxContourCenter(black_line , frame)

### Using Thresholding

In [129]:
def thresh(frame):
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray,5)

    res = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    cv2.imshow('thresh', res)
    

In [130]:
cap = cv2.VideoCapture(0)
cap.open("http://192.168.1.3:8080/video")

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    frame = cv2.rotate( frame, cv2.ROTATE_90_CLOCKWISE)
    frame = cv2.resize(frame, (640, 480))
    
    # canny(frame)
    # houghLines(frame)
    detect_black(frame)
    # thresh(frame)

    cv2.imshow('frame', frame)
    key = cv2.waitKey(1) & 0xFF	
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()